In [181]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read Data and get basic county groups

0.0 Report Data

In [182]:
df_world_original = pd.read_csv("WHO-COVID-19-global-data.csv")
df_world_original

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-02-24T00:00:00Z,AF,Afghanistan,EMRO,1,1,0,0
1,2020-02-25T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
2,2020-02-26T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
3,2020-02-27T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
4,2020-02-28T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
...,...,...,...,...,...,...,...,...
18265,2020-05-25T00:00:00Z,ZW,Zimbabwe,AFRO,0,56,0,4
18266,2020-05-26T00:00:00Z,ZW,Zimbabwe,AFRO,0,56,0,4
18267,2020-05-27T00:00:00Z,ZW,Zimbabwe,AFRO,0,56,0,4
18268,2020-05-28T00:00:00Z,ZW,Zimbabwe,AFRO,76,132,0,4


In [183]:
df_world_original[df_world_original['Date_reported']=='2020-05-29T00:00:00Z']['Cumulative_cases'].sum()

5701337

In [184]:
df_world_original[df_world_original['Date_reported']=='2020-05-29T00:00:00Z']['Cumulative_deaths'].sum()

357688

In [185]:
df_world_original.loc[df_world_original['Country']=='Namibia','Country_code'] = 'NA'
df_world_original[df_world_original['Country']=='Namibia']

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
11435,2020-03-14T00:00:00Z,NA,Namibia,AFRO,2,2,0,0
11436,2020-03-15T00:00:00Z,NA,Namibia,AFRO,0,2,0,0
11437,2020-03-16T00:00:00Z,NA,Namibia,AFRO,0,2,0,0
11438,2020-03-17T00:00:00Z,NA,Namibia,AFRO,0,2,0,0
11439,2020-03-18T00:00:00Z,NA,Namibia,AFRO,0,2,0,0
...,...,...,...,...,...,...,...,...
11507,2020-05-25T00:00:00Z,NA,Namibia,AFRO,1,21,0,0
11508,2020-05-26T00:00:00Z,NA,Namibia,AFRO,0,21,0,0
11509,2020-05-27T00:00:00Z,NA,Namibia,AFRO,0,21,0,0
11510,2020-05-28T00:00:00Z,NA,Namibia,AFRO,1,22,0,0


In [186]:
df_world_original[df_world_original['Country_code'].isna()==True]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths


In [187]:
df_world_original = df_world_original.drop(columns=['Country','WHO_region','New_cases','New_deaths'])
df_world_original

,Date_reported,Country_code,Cumulative_cases,Cumulative_deaths
0,2020-02-24T00:00:00Z,AF,1,0
1,2020-02-25T00:00:00Z,AF,1,0
2,2020-02-26T00:00:00Z,AF,1,0
3,2020-02-27T00:00:00Z,AF,1,0
4,2020-02-28T00:00:00Z,AF,1,0
...,...,...,...,...
18265,2020-05-25T00:00:00Z,ZW,56,4
18266,2020-05-26T00:00:00Z,ZW,56,4
18267,2020-05-27T00:00:00Z,ZW,56,4
18268,2020-05-28T00:00:00Z,ZW,132,4


In [188]:
df_world_original = df_world_original.rename(columns={"Date_reported": "date", "Country_code": "country_code", 
                                                      "Cumulative_cases": "cases", "Cumulative_deaths": "deaths"})
df_world_original

,date,country_code,cases,deaths
0,2020-02-24T00:00:00Z,AF,1,0
1,2020-02-25T00:00:00Z,AF,1,0
2,2020-02-26T00:00:00Z,AF,1,0
3,2020-02-27T00:00:00Z,AF,1,0
4,2020-02-28T00:00:00Z,AF,1,0
...,...,...,...,...
18265,2020-05-25T00:00:00Z,ZW,56,4
18266,2020-05-26T00:00:00Z,ZW,56,4
18267,2020-05-27T00:00:00Z,ZW,56,4
18268,2020-05-28T00:00:00Z,ZW,132,4


In [189]:
type(df_world_original.loc[0,'date'])

str

In [190]:
df_world_original['date'] = df_world_original['date'].str.strip('T00:00:00Z')
df_world_original

,date,country_code,cases,deaths
0,2020-02-24,AF,1,0
1,2020-02-25,AF,1,0
2,2020-02-26,AF,1,0
3,2020-02-27,AF,1,0
4,2020-02-28,AF,1,0
...,...,...,...,...
18265,2020-05-25,ZW,56,4
18266,2020-05-26,ZW,56,4
18267,2020-05-27,ZW,56,4
18268,2020-05-28,ZW,132,4


0.1 GeoJSON Data

In [191]:
old_world_geojson_df = gpd.read_file("World_Countries_Boundaries_new.geojson")
old_world_geojson_df

,OBJECTID,FIPS_CNTRY,ISO_2DIGIT,ISO_3DIGIT,NAME,LONG_NAME,LOCSHRTNAM,LOCLNGNAM,CAPITAL,COUNTRYAFF,CONTINENT,COLORMAP,Shape_Leng,Shape_Area,geometry
0,1,AQ,AS,ASM,American Samoa,Territory of American Samoa,None,None,Pago Pago,United States,Oceania,2,0.600124,0.013720,"POLYGON ((-170.744 -14.376, -170.823 -14.324, ..."
1,2,WQ,UM,UMI,United States Minor Outlying Islands,None,None,None,None,United States,Oceania,7,0.028875,0.000034,"MULTIPOLYGON (((-160.021 -0.398, -160.043 -0.3..."
2,3,CW,CK,COK,Cook Islands,Cook Islands,None,None,Avarua,New Zealand,Oceania,6,0.980664,0.013073,"MULTIPOLYGON (((-159.747 -21.257, -159.833 -21..."
3,4,FP,PF,PYF,French Polynesia,Overseas Lands of French Polynesia,Polynesie Francaise,Pays d'outre-mer de la Polynesie Francaise,Papeete,France,Oceania,4,3.930211,0.175332,"MULTIPOLYGON (((-149.179 -17.871, -149.276 -17..."
4,7,NE,NU,NIU,Niue,Niue,None,None,Alofi,New Zealand,Oceania,2,0.541413,0.021414,"POLYGON ((-169.894 -19.146, -169.931 -19.124, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,249,FM,FM,FSM,Micronesia,Federated States of Micronesia,None,None,Palikir,Micronesia,Oceania,2,1.042421,0.042206,"MULTIPOLYGON (((158.228 6.781, 158.155 6.819, ..."
240,250,CQ,MP,MNP,Northern Mariana Islands,Commonwealth of the Northern Mariana Islands,None,None,Saipan,United States,Oceania,1,0.908853,0.019927,"MULTIPOLYGON (((145.735 15.087, 145.685 15.101..."
241,251,PS,PW,PLW,Palau,Republic of Palau,Belau,Beluu er a Belau,Melekeok,Palau,Oceania,1,1.105323,0.031136,"MULTIPOLYGON (((134.531 7.354, 134.485 7.438, ..."
242,253,RS,RU,RUS,Russian Federation,Russian Federation,Rossiya,Rossiyskaya Federatsiya,Moscow,Russian Federation,Asia,1,1526.025072,2929.641772,"MULTIPOLYGON (((131.873 42.957, 131.824 42.953..."


In [192]:
old_world_geojson_df = old_world_geojson_df[['ISO_2DIGIT','NAME','geometry']]
old_world_geojson_df

,ISO_2DIGIT,NAME,geometry
0,AS,American Samoa,"POLYGON ((-170.744 -14.376, -170.823 -14.324, ..."
1,UM,United States Minor Outlying Islands,"MULTIPOLYGON (((-160.021 -0.398, -160.043 -0.3..."
2,CK,Cook Islands,"MULTIPOLYGON (((-159.747 -21.257, -159.833 -21..."
3,PF,French Polynesia,"MULTIPOLYGON (((-149.179 -17.871, -149.276 -17..."
4,NU,Niue,"POLYGON ((-169.894 -19.146, -169.931 -19.124, ..."
...,...,...,...
239,FM,Micronesia,"MULTIPOLYGON (((158.228 6.781, 158.155 6.819, ..."
240,MP,Northern Mariana Islands,"MULTIPOLYGON (((145.735 15.087, 145.685 15.101..."
241,PW,Palau,"MULTIPOLYGON (((134.531 7.354, 134.485 7.438, ..."
242,RU,Russian Federation,"MULTIPOLYGON (((131.873 42.957, 131.824 42.953..."


In [193]:
#Add Kosovo geometry
import shapely.wkt
polygon1 = shapely.wkt.loads('POLYGON ((21.160269147746988 42.66353984291385, 21.160269147746988 42.6571648755645, 21.169109708660073 42.6571648755645, 21.169109708660073 42.66353984291385, 21.160269147746988 42.66353984291385))')
old_world_geojson_df = old_world_geojson_df.append({'ISO_2DIGIT':'XK', 'NAME':'Kosovo', 'geometry': polygon1}, ignore_index=True)

In [194]:
#Add OTHERS geometry
import shapely.wkt
polygon2 = shapely.wkt.loads('POLYGON ((6.124162972753766 46.22138299367113, 6.124162972753766 46.20594098233243, 6.1473372586424375 46.20594098233243, 6.1473372586424375 46.22138299367113, 6.124162972753766 46.22138299367113))')
old_world_geojson_df = old_world_geojson_df.append({'ISO_2DIGIT':'Other*', 'NAME':'Others', 'geometry': polygon2}, ignore_index=True)

In [195]:
old_world_geojson_df

,ISO_2DIGIT,NAME,geometry
0,AS,American Samoa,"POLYGON ((-170.744 -14.376, -170.823 -14.324, ..."
1,UM,United States Minor Outlying Islands,"MULTIPOLYGON (((-160.021 -0.398, -160.043 -0.3..."
2,CK,Cook Islands,"MULTIPOLYGON (((-159.747 -21.257, -159.833 -21..."
3,PF,French Polynesia,"MULTIPOLYGON (((-149.179 -17.871, -149.276 -17..."
4,NU,Niue,"POLYGON ((-169.894 -19.146, -169.931 -19.124, ..."
...,...,...,...
241,PW,Palau,"MULTIPOLYGON (((134.531 7.354, 134.485 7.438, ..."
242,RU,Russian Federation,"MULTIPOLYGON (((131.873 42.957, 131.824 42.953..."
243,ES,Spain,"MULTIPOLYGON (((-17.911 27.774, -17.983 27.638..."
244,XK,Kosovo,"POLYGON ((21.160 42.664, 21.160 42.657, 21.169..."


In [196]:
list(set(df_world_original['country_code'].unique())-set(old_world_geojson_df['ISO_2DIGIT'].unique()))

[]

Group Data to world

In [197]:
pivot_world = pd.pivot_table(df_world_original , index=['country_code'], columns=['date'])
pivot_world = pivot_world.replace(np.nan, 0)
pivot_world

cases                                              \
date         2020-01-11 2020-01-12 2020-01-13 2020-01-14 2020-01-15   
country_code                                                          
AD                  0.0        0.0        0.0        0.0        0.0   
AE                  0.0        0.0        0.0        0.0        0.0   
AF                  0.0        0.0        0.0        0.0        0.0   
AG                  0.0        0.0        0.0        0.0        0.0   
AI                  0.0        0.0        0.0        0.0        0.0   
...                 ...        ...        ...        ...        ...   
YE                  0.0        0.0        0.0        0.0        0.0   
YT                  0.0        0.0        0.0        0.0        0.0   
ZA                  0.0        0.0        0.0        0.0        0.0   
ZM                  0.0        0.0        0.0        0.0        0.0   
ZW                  0.0        0.0        0.0        0.0        0.0   

                                                                    ...  \
date         2020-01-16 2020-01-17 2020-01-18 2020-01-19 2020-01-2  ...   
country_code                                                        ...   
AD                  0.0        0.0        0.0        0.0       0.0  ...   
AE                  0.0        0.0        0.0        0.0       0.0  ...   
AF                  0.0        0.0        0.0        0.0       0.0  ...   
AG                  0.0        0.0        0.0        0.0       0.0  ...   
AI                  0.0        0.0        0.0        0.0       0.0  ...   
...                 ...        ...        ...        ...       ...  ...   
YE                  0.0        0.0        0.0        0.0       0.0  ...   
YT                  0.0        0.0        0.0        0.0       0.0  ...   
ZA                  0.0        0.0        0.0        0.0       0.0  ...   
ZM                  0.0        0.0        0.0        0.0       0.0  ...   
ZW                  0.0        0.0        0.0        0.0       0.0  ...   

                deaths                                                         \
date         2020-05-2 2020-05-21 2020-05-22 2020-05-23 2020-05-24 2020-05-25   
country_code                                                                    
AD                51.0       51.0       51.0       51.0       51.0       51.0   
AE               227.0      233.0      237.0      241.0      244.0      245.0   
AF               187.0      193.0      205.0      211.0      218.0      219.0   
AG                 3.0        3.0        3.0        3.0        3.0        3.0   
AI                 0.0        0.0        0.0        0.0        0.0        0.0   
...                ...        ...        ...        ...        ...        ...   
YE                29.0       30.0       33.0       34.0       40.0       43.0   
YT                19.0       19.0       19.0       19.0       19.0       20.0   
ZA               312.0      339.0      369.0      397.0      407.0      429.0   
ZM                 7.0        7.0        7.0        7.0        7.0        7.0   
ZW                 4.0        4.0        4.0        4.0        4.0        4.0   

                                                          
date         2020-05-26 2020-05-27 2020-05-28 2020-05-29  
country_code                                              
AD                 51.0       51.0       51.0       51.0  
AE                248.0      253.0      255.0      258.0  
AF                220.0      226.0      235.0      246.0  
AG                  3.0        3.0        3.0        3.0  
AI                  0.0        0.0        0.0        0.0  
...                 ...        ...        ...        ...  
YE                 45.0       50.0       54.0       61.0  
YT                 20.0       20.0       20.0       21.0  
ZA                481.0      524.0      552.0      577.0  
ZM                  7.0        7.0        7.0        7.0  
ZW                  4.0        4.0        4.0        4.0  

[216 rows 

0.3 Get Date Info

In [198]:
date= pivot_world['cases'].columns
date

Index(['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15',
       '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-2',
       ...
       '2020-05-2', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24',
       '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29'],
      dtype='object', name='date', length=140)

In [199]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [200]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

140 DatetimeIndex(['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14',
               '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18',
               '2020-01-19', '2020-01-20',
               ...
               '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23',
               '2020-05-24', '2020-05-25', '2020-05-26', '2020-05-27',
               '2020-05-28', '2020-05-29'],
              dtype='datetime64[ns]', length=140, freq='D')
140 ['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [201]:
# import json
# pivot_world['cases_ts'] = json.dumps({"values": pivot_world['cases'].values.tolist()[0]})
# pivot_world['deaths_ts'] =  json.dumps({"values": pivot_world['deaths'].values.tolist()[0]})

pivot_world['cases_ts'] =  pivot_world['cases'].values.tolist()
pivot_world['deaths_ts'] =  pivot_world['deaths'].values.tolist()

Form Dynamic classes for new_case and new_death 

In [202]:
new_case_dict = {}
for i in range(1,len(date)):
    q6 = mc.Quantiles(pivot_world['cases_ts'].apply(lambda x: x[i] - x[i - 1]).values, k=6)
    new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 1
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:890: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 2
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 3
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaco

In [203]:
dynamic_cl_dict = {}
dynamic_cl_dict['county'] = {'new_case':new_case_dict, 'new_death':{}}
dynamic_cl_dict['state'] = {}

In [204]:
import json
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_cl_dict, json_file)

In [205]:
pivot_world['dt_first_case'] = (pivot_world['cases'] > 0).idxmax(axis=1)
pivot_world['dt_first_death'] = (pivot_world['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths world
pivot_world.loc[pivot_world['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_world.head(5)

cases                                              \
date         2020-01-11 2020-01-12 2020-01-13 2020-01-14 2020-01-15   
country_code                                                          
AD                  0.0        0.0        0.0        0.0        0.0   
AE                  0.0        0.0        0.0        0.0        0.0   
AF                  0.0        0.0        0.0        0.0        0.0   
AG                  0.0        0.0        0.0        0.0        0.0   
AI                  0.0        0.0        0.0        0.0        0.0   

                                                                    ...  \
date         2020-01-16 2020-01-17 2020-01-18 2020-01-19 2020-01-2  ...   
country_code                                                        ...   
AD                  0.0        0.0        0.0        0.0       0.0  ...   
AE                  0.0        0.0        0.0        0.0       0.0  ...   
AF                  0.0        0.0        0.0        0.0       0.0  ...   
AG                  0.0        0.0        0.0        0.0       0.0  ...   
AI                  0.0        0.0        0.0        0.0       0.0  ...   

                 deaths                                              \
date         2020-05-24 2020-05-25 2020-05-26 2020-05-27 2020-05-28   
country_code                                                          
AD                 51.0       51.0       51.0       51.0       51.0   
AE                244.0      245.0      248.0      253.0      255.0   
AF                218.0      219.0      220.0      226.0      235.0   
AG                  3.0        3.0        3.0        3.0        3.0   
AI                  0.0        0.0        0.0        0.0        0.0   

                                                                  cases_ts  \
date         2020-05-29                                                      
country_code                                                                 
AD                 51.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
AE                258.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
AF                246.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
AG                  3.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
AI                  0.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                      deaths_ts dt_first_case  \
date                                                                            
country_code                                                                    
AD            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-03-02   
AE            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-01-29   
AF            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-02-24   
AG            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-03-13   
AI            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-03-26   

             dt_first_death  
date                         
country_code                 
AD               2020-03-24  
AE               2020-03-21  
AF               2020-03-23  
AG               2020-04-09  
AI                      NaN  

[5 rows x 284 columns]

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [206]:
yesterday = date.values[-2]
pivot_world['today_case'] = pivot_world['cases'][dt_str_end]
pivot_world['today_new_case'] = pivot_world['cases'][dt_str_end] - pivot_world['cases'][yesterday]
pivot_world['today_death'] = pivot_world['deaths'][dt_str_end]
pivot_world['today_new_death'] = pivot_world['deaths'][dt_str_end] - pivot_world['deaths'][yesterday]

Step3:  
Extract necessary columns

In [207]:
report_df = pivot_world[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['country_code','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,country_code,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,AD,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-02,2020-03-24,764.0,1.0,51.0,0.0
1,AE,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-01-29,2020-03-21,32532.0,563.0,258.0,3.0
2,AF,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-24,2020-03-23,13659.0,643.0,246.0,11.0
3,AG,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-13,2020-04-09,25.0,0.0,3.0,0.0
4,AI,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-26,NaN,3.0,0.0,0.0,0.0


Step4:  
Assign Geometries to world

In [208]:
final_df = pd.merge(old_world_geojson_df, report_df, how='left', left_on=['ISO_2DIGIT'], right_on = ['country_code'])
final_df.columns

Index(['ISO_2DIGIT', 'NAME', 'geometry', 'country_code', 'cases_ts',
       'deaths_ts', 'dt_first_case', 'dt_first_death', 'today_case',
       'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

In [209]:
final_df

,ISO_2DIGIT,NAME,geometry,country_code,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,AS,American Samoa,"POLYGON ((-170.744 -14.376, -170.823 -14.324, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UM,United States Minor Outlying Islands,"MULTIPOLYGON (((-160.021 -0.398, -160.043 -0.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CK,Cook Islands,"MULTIPOLYGON (((-159.747 -21.257, -159.833 -21...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PF,French Polynesia,"MULTIPOLYGON (((-149.179 -17.871, -149.276 -17...",PF,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-12,NaN,60.0,0.0,0.0,0.0
4,NU,Niue,"POLYGON ((-169.894 -19.146, -169.931 -19.124, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
241,PW,Palau,"MULTIPOLYGON (((134.531 7.354, 134.485 7.438, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,RU,Russian Federation,"MULTIPOLYGON (((131.873 42.957, 131.824 42.953...",RU,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-01-31,2020-03-26,387623.0,8572.0,4374.0,232.0
243,ES,Spain,"MULTIPOLYGON (((-17.911 27.774, -17.983 27.638...",ES,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-01-31,2020-03-04,238278.0,1137.0,29037.0,1.0
244,XK,Kosovo,"POLYGON ((21.160 42.664, 21.160 42.657, 21.169...",XK,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-13,2020-03-23,1048.0,1.0,30.0,0.0


Step5:  
Deal with nan values (world not shows up in geojson) (More works)

In [210]:
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)

In [211]:
template =final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]

In [212]:
for x in range(0,len(template)):
    template[x] = 0
template = ','.join(map(str, template))

In [213]:
final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df.head(5)

,ISO_2DIGIT,NAME,geometry,country_code,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,AS,American Samoa,"POLYGON ((-170.74390 -14.37555, -170.82323 -14...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
1,UM,United States Minor Outlying Islands,"MULTIPOLYGON (((-160.02114 -0.39805, -160.0434...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
2,CK,Cook Islands,"MULTIPOLYGON (((-159.74698 -21.25667, -159.832...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
3,PF,French Polynesia,"MULTIPOLYGON (((-149.17920 -17.87084, -149.275...",PF,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-12,NaN,60.0,0.0,0.0,0.0
4,NU,Niue,"POLYGON ((-169.89389 -19.14556, -169.93088 -19...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0


In [214]:
final_df

,ISO_2DIGIT,NAME,geometry,country_code,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,AS,American Samoa,"POLYGON ((-170.744 -14.376, -170.823 -14.324, ...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
1,UM,United States Minor Outlying Islands,"MULTIPOLYGON (((-160.021 -0.398, -160.043 -0.3...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
2,CK,Cook Islands,"MULTIPOLYGON (((-159.747 -21.257, -159.833 -21...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
3,PF,French Polynesia,"MULTIPOLYGON (((-149.179 -17.871, -149.276 -17...",PF,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-12,NaN,60.0,0.0,0.0,0.0
4,NU,Niue,"POLYGON ((-169.894 -19.146, -169.931 -19.124, ...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
241,PW,Palau,"MULTIPOLYGON (((134.531 7.354, 134.485 7.438, ...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
242,RU,Russian Federation,"MULTIPOLYGON (((131.873 42.957, 131.824 42.953...",RU,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-01-31,2020-03-26,387623.0,8572.0,4374.0,232.0
243,ES,Spain,"MULTIPOLYGON (((-17.911 27.774, -17.983 27.638...",ES,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-01-31,2020-03-04,238278.0,1137.0,29037.0,1.0
244,XK,Kosovo,"POLYGON ((21.160 42.664, 21.160 42.657, 21.169...",XK,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-13,2020-03-23,1048.0,1.0,30.0,0.0


In [215]:
print(final_df['ISO_2DIGIT'].value_counts().head(10))

VC    1
LY    1
TH    1
FJ    1
VI    1
GY    1
EG    1
DE    1
WF    1
YE    1
Name: ISO_2DIGIT, dtype: int64


In [216]:
final_df['ISO_2DIGIT']

0          AS
1          UM
2          CK
3          PF
4          NU
        ...  
241        PW
242        RU
243        ES
244        XK
245    Other*
Name: ISO_2DIGIT, Length: 246, dtype: object

In [217]:
final_df['ISO_2DIGIT'].value_counts()

VC    1
LY    1
TH    1
FJ    1
VI    1
     ..
TJ    1
SX    1
MT    1
BA    1
GW    1
Name: ISO_2DIGIT, Length: 246, dtype: int64

Step6:  
Finalize the Dataframe

In [218]:
final_df['dt_start'] = dt_str_start
final_df['dt_end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df['population'] = 0
final_df.head(1)

,ISO_2DIGIT,NAME,geometry,country_code,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,population
0,AS,American Samoa,"POLYGON ((-170.74390 -14.37555, -170.82323 -14...",NaN,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0,2020-01-11,2020-05-29,day,0


Step 7:  
Output file

In [219]:
final_df.to_file("who_world_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done
